In [ ]:

import os 
openai_api_key = os.environ.get('OPENAI_API_KEY')

testing model 3.5 turbo

In [ ]:
from langchain import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
text = input("Enter text: ")
print(llm(text))

Chat Model

a model that take series of messages and return a message output

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(temperature=1, openai_api_key=openai_api_key)
chat([
    SystemMessage(content="You are a gross bot that makes every word gross."),
    HumanMessage(content="Is Benzema going to Al Ittihad?")
])

Prompt Template

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo",temperature=0.1)

chat = """
Where is {football_player} playing right now?
"""

prompt = PromptTemplate(
    input_variables=["football_player"],
    template = chat
)

text = prompt.format(football_player=input("Enter football player: "))
print(llm(text))

Example Selectors:
select a series of examples that allow you to dynamic place in-context info into your prompt.

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key=openai_api_key,temperature=0.1, model_name="gpt-3.5-turbo-0613")

# give example 
example_prompt = PromptTemplate(
    input_variables=["football_player", "team"],
    template = "Example Football Player: {football_player}\nCurrent Team: {team}"
)

examples = [
    {"football_player": "Benzema", "team": "Real Madrid"},
    {"football_player": "Messi", "team": "Barcelona"},
    {"football_player": "Ronaldo", "team": "Juventus"},
    {"football_player": "Mbappe", "team": "PSG"},
    {"football_player": "Neymar", "team": "PSG"},
    {"football_player": "Haaland", "team": "Dortmund"},
    {"football_player": "Lewandowski", "team": "Bayern Munich"},
    {"football_player": "Kane", "team": "Tottenham"},
    {"football_player": "Salah", "team": "Liverpool"},
    {"football_player": "Beckham", "team": "Retired"},
    {"football_player": "Zidane", "team": "Retired"},
    {"football_player": "Ronaldinho", "team": "Retired"},
    {"football_player": "Cafu", "team": "Retired"},
]

# SemanticSimilarityExampleSelector will select example that are similar to input
example_selector = SemanticSimilarityExampleSelector.from_examples(

    # list of examples available to select from 
    examples,
    
    # embedding
    OpenAIEmbeddings(openai_api_key=openai_api_key),
    
    # vector store embeddings and do a similar
    FAISS,
    
    # how many we want
    k=1,
)

similar_prompt = FewShotPromptTemplate(
    # this will help select examples
    example_selector=example_selector,
    example_prompt=example_prompt,

    # customize that will be added to the top of bottom of your prompt
    prefix="give the club of football player plays in",
    suffix="football_player: {football_player}\nteam:",

    # what inputs your prompt will receive

    input_variables=["football_player"],
)

my_player = input("Enter football player: ")
result = llm(similar_prompt.format(football_player=my_player))
print(f"Football Player: {my_player}\nTeam: {result}")


Output Parser: 
format model into structured output like JSON.

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-0613", temperature=.1)

# How would you like your response structured. A fancy prompt template
response_schemas = [
    ResponseSchema(name="bad_string", description="Poorly formatted user input string"),
    ResponseSchema(name="good_string", description="Well formatted user input string"),
]

#  How would you like to parse the output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print(format_instructions)


In [ ]:
template = """
You will be given a poorly formatted string from a user. Reformat it and make sure all the words are spelled correctly.

{format_instructions}

% USER INPUT
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions}, 
    template=template
)

promptValue = prompt.format(user_input="batma is brce wyn")
print(promptValue)

In [ ]:
llm_output = llm(promptValue)
llm_output

In [ ]:
output_parser.parse(llm_output)

Indexes - Structuring documents to LLMs can work with them

Doucments Loaders 
Easy way to import data from other sources. Shared functionality with OpenAI Plugins


In [ ]:
from langchain.document_loaders import HNLoader
loader = HNLoader("https://www.reddit.com/r/antiwork/comments/14feib2/i_mean_i_need_to_sleep_as_well/")
data = loader.load()
print(f"Found {len(data)} comments")

Text Splitters
when your document is too long like a book for LLM. need to split it to chunks. Text splitters help with this.
There are many ways you could split.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 
with open('C:/Users/USER/OneDrive/Documents/ai-chatbot-3/server/test_data/test.txt') as f:
    pg_work = f.read()
print(f"You have {len(pg_work)} documents")


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=20,
)

text = text_splitter.create_documents(pg_work)
print(f"You have {len(text)} documents")


Retrievers: Bind Documents with LLM. Vectorstore retriever

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader("C:/Users/USER/OneDrive/Documents/ai-chatbot-3/server/test_data/test.txt")
document = loader.load()

In [ ]:
# split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
)

# split doc into texts
texts = text_splitter.split_documents(document)

# create embeddings
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)

# embed text
db = FAISS.from_documents(texts, embedding)

In [ ]:
retriever = db.as_retriever()
docs = retriever.get_relevant_documents("When was Software Engineering first coined?")
print("\n\n".join([x.page_content[:200] for x in docs[:2]]))

Memory:

Help LLM remember information.
Memory is a bit of a loose term. It could be simple as remembering what you chatted about. 

we keep it towards the chat message use case. this will be used for chat bots. 

Chat Message History

In [ ]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    openai_api_key=openai_api_key,
    temperature=0.1,
    model_name="gpt-3.5-turbo-0613",
)

history = ChatMessageHistory()

history.add_ai_message("Hello, how are you?")

history.add_user_message("I am doing well, how are you?")


In [ ]:
history.messages

In [ ]:
ai_response = chat(history.messages)
ai_response


In [ ]:
history.add_ai_message(ai_response.content)
history.messages

Chains
combining LLMs calls and actions automatically. 

1. Simple Sequemtial Chains
    easy chains where you can use the output of an LLM as an input into another. Good for breaking up tasks and keep it focused.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import SimpleSequentialChain, LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0613", temperature=.1, streaming=True)

In [ ]:
template = """
Mention a football club that the player that user mentions is in his prime in.

%USER PLAYER
{user_player}

YOUR RESPONSE:
"""
prompt_template  = PromptTemplate(
    input_variables=["user_player"],
    template=template
)

player_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
)

In [ ]:
template = """
Given a football club, give a name of the famous player and give info pf his G/A in how many appearances for the club.

%USER CLUB
{user_club}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(
    input_variables=["user_club"],
    template=template
)

club_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
)

In [ ]:
overall_chain = SimpleSequentialChain(
    chains=[player_chain, club_chain]
)


In [ ]:
review = overall_chain.run(input("Enter name of football player or club: "))
print(review)

Economist and Business Analyst

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import SimpleSequentialChain, LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0613", temperature=.1,)


In [ ]:
template = """
You are a professional economist. You share your opinion on the topic that user provides in an economist perspective.

%USER TOPIC
{user_topic}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(
    input_variables=["user_topic"],
    template=template
)

economist_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
)

In [ ]:
template = """
You are a professional business analyst. You share your opinion(sometimes agree, sometimes do not agree) on what the economist said about the topic user provides in an analyst perspective.

% ECONOMIST SAID
{economist_said}
"""
prompt_template = PromptTemplate(
    input_variables=["economist_said"],
    template=template
)

business_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
)


In [ ]:
overall_chain = SimpleSequentialChain(
    chains=[economist_chain, business_chain],
    verbose=True,
)
user_input = input("Enter topic: ")
review = overall_chain.run(user_input)
print(f"Economist: {economist_chain.run(user_input)}")
print(f"Business Analyst: {review}")

Summerization Chain: 
easily run thru long numerous chain and get a summary.

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0613", temperature=.1)

loader = TextLoader('C:/Users/USER/OneDrive/Documents/ai-chatbot-3/server/test_data/test.txt')
documents = loader.load()

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=50,
)

# Split your docs into text
texts = text_splitter.split_documents(documents)

# Load your summarizer
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts)


Agents:
The language model that drives the decision making.

it takes in an input and returns a response to an action to take along with an action input. You can see different types of agents which are better for different use cases.

Tools
a capibility of an agent. This is an abstraction on top of a function that makes it easy to for LLMs and agents to interact with it EX: Goolge Search

In [1]:
from langchain.agents import load_tools, initialize_agent
from langchain.chat_models import ChatOpenAI
import json

In [2]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0613", temperature=.1)
serpapi_api_key = "10f2e4c2873c46a9a7cb3bb806af9d7688f141e8f2ac293422d971b48225b136"


In [4]:
toolkit = load_tools(['serpapi'], llm=llm, serpapi_api_key=serpapi_api_key)
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)
response = agent({
    "input": input("Enter: "),
}) 



> Entering new AgentExecutor chain...
I should search for the current price of Bitcoin.
Action: Search
Action Input: "current price of Bitcoin"
Observation: The Bitcoin price is $30,460.13, a change of -0.34% over the past 24 hours as of 12:00 a.m. The recent price action in Bitcoin left the tokens market ...
Thought:I now know the current price of Bitcoin.
Final Answer: The current price of Bitcoin is $30,460.13.

> Finished chain.


In [ ]:
print(json.dumps(response["intermediate_steps"], indent=2))